In [1]:
# === Инициализация секретов из окружения (без setdefault) ===
import os, pathlib
import streamlit as st  # pip install streamlit

class _Secrets(dict): pass

key = os.environ.get("OPENAI_API_KEY")
if not key:
    raise RuntimeError("OPENAI_API_KEY не найден в окружении. Экспортируй его перед запуском ноутбука.")

# эмулируем st.secrets для rag_agent
st.secrets = _Secrets({"OPENAI_API_KEY": key})

# создаём папку для данных, если её нет
data_dir = pathlib.Path("data/uploads")
data_dir.mkdir(parents=True, exist_ok=True)

print("OK: ключ взят из окружения. Папка для данных:", data_dir)


OK: ключ взят из окружения. Папка для данных: data/uploads


In [2]:
# === Переиндексация корпуса ===
import rag_agent  # твой файл с агентом
total = rag_agent.reindex_corpus()
print("Документов в индексе:", total)


/Users/DDV/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/DDV/Desktop/синергия/Lobov/rag_agent.py:205: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  _embeddings = OpenAIEmbeddings(
/Users/DDV/Library/Python/3.9/lib/python/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Документов в индексе: 1909


In [4]:
# === Тестовый запрос ===
chain = rag_agent.build_rag_chain()

def ask(q: str):
    out = chain(q)
    print("Q:", q, "\n")
    print("A:", out["result"], "\n")
    print("— Источники —")
    for i, d in enumerate(out.get("source_documents", []), 1):
        m = d.metadata or {}
        print(f"[{i}] {m.get('label')} | дата: {m.get('date')} | id: {m.get('doc_id')} | file: {m.get('source')}")
    print("="*80)

ask("Какие ключевые инициативы объявляли на брифингах в июле 2024 и что они содержали?")


Q: Какие ключевые инициативы объявляли на брифингах в июле 2024 и что они содержали? 

A: Из предоставленных материалов невозможно ответить на ваш вопрос, так как они не содержат информации о брифингах, проведенных в июле 2024 года и объявленных на них инициативах.

Источники:
1. Доклад ЛВГ _ ССП 2025.pdf, страница 1, data/uploads/all_presentations.json
2. Мастер-класс ЛВГ.pdf, страница 36, data/uploads/all_presentations.json 

— Источники —
[1] Доклад ЛВГ _ ССП 2025.pdf — стр. 1 | дата: неизвестно | id: bf23402e19 | file: data/uploads/all_presentations.json
[2] Мастер-класс ЛВГ.pdf — стр. 104 | дата: неизвестно | id: de348edffa | file: data/uploads/all_presentations.json
[3] Мастер-класс ЛВГ.pdf — стр. 36 | дата: неизвестно | id: b27d411e80 | file: data/uploads/all_presentations.json
[4] Документ №10 | дата: 02.04.2025 | id: 10 | file: meeting_summaries.csv


In [5]:
ask("Какие были брифинги у лобова?")

Q: Какие были брифинги у лобова? 

A: Были проведены как минимум три брифинга под руководством Лобова. На первом брифинге Лобов дал ряд поручений своим сотрудникам: Медведевой - собрать контакты и организовать приглашения для ключевых депутатов и сенаторов, Кучерявому - подготовить доклад по проекту «Защитник», Поляковой - проработать вопрос квот на привоз иностранных работников и подготовить предложения, Андрею - войти в состав союза. На втором и третьем брифингах конкретные детали не были указаны, но было упомянуто, что они включали в себя представление новых руководителей.

Источники:
1. "[Russian (auto-generated)] Видео с первого брифинга Президента Корпорации - В.Г. Лобова. [DownSub.com].txt", дата неизвестна.
2. "[Russian (auto-generated)] Второй брифинг Президента Корпорации Вадима Лобова [DownSub.com].txt", дата неизвестна. 

— Источники —
[1] Документ №28 | дата: 26.03.2025 | id: 28 | file: meeting_summaries.csv
[2] [Russian (auto-generated)] Видео с первого брифинга Президент